# Stats changes

Look at how stats change over time

## Imports

In [ ]:
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import xarray as xr
import tqdm
import pathlib
import cmocean
import os
import src.XRO
import copy
import scipy.stats
import warnings
import calendar
import pandas as pd
import cartopy.util

# import gsw

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False}, palette="colorblind")

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## set style specs
ax_kwargs = dict(ls="--", c="gray", lw=0.8)

## Load data

Function to get windowed data

In [ ]:
def get_windowed(x):
    return src.utils.get_windowed(x, stride=120)

In [ ]:
## load spatial data
forced, anom = src.utils.load_consolidated()

## load Th data
Th = src.utils.load_cesm_indices()

## load ELI
eli = xr.open_dataset(pathlib.Path(DATA_FP, "cesm/eli.nc"))
_, eli_anom = src.utils.separate_forced(eli)

## merge
Th = xr.merge([Th, eli_anom])

Compute relative SST

In [ ]:
## load tropical SST data and total data
Th_total = xr.open_dataset(DATA_FP / "cesm" / "Th.nc")
trop_sst = xr.open_dataset(pathlib.Path(DATA_FP, "cesm/trop_sst.nc"))

for v in ["T_3", "T_34", "T_4"]:
    Th[f"{v}_rel"] = Th_total[v] - trop_sst["trop_sst_15"]

Get windowed data

In [ ]:
## merge, and get windowed data
Th = get_windowed(Th)

## Analysis

### Heat flux

In [ ]:
prQ_n34 = get_windowed(
    src.utils.reconstruct_wrapper(
        anom[["pr", "pr_comp", "nhf", "nhf_comp"]],
        fn=src.utils.get_nino34,
    ),
)

Plot data from all years

In [ ]:
## plot var
PLOT_VAR = "nhf"

## get plot data
sel = lambda x: src.utils.sel_month(x, 12).transpose("time", "member", ...)
# x0 = sel(Th["T_34_rel"])
x0 = sel(Th["T_34_rel"])
x1 = sel(prQ_n34[PLOT_VAR])

## make plot
fig, ax = plt.subplots(1, 1, figsize=(3, 2.5), layout="constrained")

# for ax, y_ in zip(axs, [1870,2030]):
ax.scatter(x0, x1, s=1, alpha=0.2)
ax.axhline(0, **ax_kwargs)
ax.axvline(0, **ax_kwargs)

# src.utils.set_lims(axs)
# axs[1].set_yticks([])
plt.show()

Plot beginning/end years

In [ ]:
## plot var
PLOT_VAR = "nhf"

## get plot data
sel = lambda x: src.utils.sel_month(x, 12).transpose("time", "member", ...)
x0 = sel(Th["T_34_rel"])
x1 = sel(prQ_n34[PLOT_VAR])

## make plot
fig, axs = plt.subplots(1, 2, figsize=(5.5, 2.5), layout="constrained")

for ax, y_ in zip(axs, [1870, 2030]):
    ax.scatter(x0.sel(year=y_), x1.sel(year=y_), s=3, alpha=0.5)
    ax.axhline(0, **ax_kwargs)
    ax.axvline(0, **ax_kwargs)

src.utils.set_lims(axs)
axs[1].set_yticks([])
plt.show()

#### Fig

In [ ]:
## plot var
PLOT_VAR = "nhf"

## get plot data
sel = lambda x: src.utils.sel_month(x, 12).transpose("time", "member", ...)
x0 = sel(Th["T_34_rel"])
x1 = sel(prQ_n34["nhf"])

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(sel(prQ_n34["pr"]).isel(year=0), sel(prQ_n34["nhf"]).isel(year=0), s=10)

In [ ]:
n_bins = 20
nhf_bins = np.linspace(-75, 40, n_bins)
sst_bins = np.linspace(-5, 3, n_bins)

get_pdf = lambda x, bins: src.utils.get_empirical_pdf(
    x.stack(samp=["time", "member"]).values, bins
)[0]

## empty array to hold PDFs
pdfs_sst = []
pdfs_nhf = []

for y_ in [1870, 1980, 2080]:
    pdfs_sst.append(get_pdf(x0.sel(year=y_), sst_bins))
    pdfs_nhf.append(get_pdf(x0.sel(year=y_), sst_bins))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6.5, 3), layout="constrained")

# for ax, y_ in zip(axs, [1870,2030]):
axs[0].scatter(x0, x1, s=1, alpha=0.2, c="k")


axs[0].axhline(0, **ax_kwargs)
for ax in axs:
    ax.axvline(0, **ax_kwargs)
    ax.set_xticks([-4, -2, 0, 2])

## plot PDFs
axs[1].stairs(pdfs_sst[0], sst_bins, fill=True, alpha=0.1, color="k", label=1870)
axs[1].stairs(pdfs_sst[1], sst_bins, lw=2, label=1980)
axs[1].stairs(pdfs_sst[2], sst_bins, lw=2, label=2080)
# axs[0].set_yticks([-75, -25, 25])
axs[1].set_yticks([])
axs[0].set_ylabel(r"$W~m^{-2}$")
axs[1].set_ylabel(r"Prob. density")
axs[0].set_xlabel(r"$^{\circ}C$")
axs[1].set_xlabel(r"$^{\circ}C$")
axs[1].yaxis.set_label_position("right")
axs[1].legend()

plt.show()